### Chatbot 

In [1]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
from utils import get_device
from models.encoder import EncoderRNN
from models.decoder import LuongAttnDecoderRNN
from models.search import GreedySearchDecoder
from config import model_config, training_config
from train import trainIters
from evaluate import evaluateInput
from vocabulary import Voc, loadPrepareData


In [ ]:
device = get_device()

# Load and prepare data
corpus_path = "data/movie-corpus/movie-corpus"
voc, pairs = loadPrepareData(corpus_path)

# Initialize word embeddings
embedding = nn.Embedding(voc.num_words, model_config['hidden_size'])

# Initialize encoder & decoder models
encoder = EncoderRNN(model_config['hidden_size'], embedding, 
                    model_config['encoder_n_layers'], model_config['dropout'])
decoder = LuongAttnDecoderRNN(model_config['attn_model'], embedding, 
                                model_config['hidden_size'], voc.num_words,
                                model_config['decoder_n_layers'], model_config['dropout'])

# Move models to device
encoder = encoder.to(device)
decoder = decoder.to(device)

# Initialize optimizers
encoder_optimizer = optim.Adam(encoder.parameters(), lr=training_config['learning_rate'])
decoder_optimizer = optim.Adam(decoder.parameters(), 
                                lr=training_config['learning_rate'] * training_config['decoder_learning_ratio'])

# Load checkpoint if exists
loadFilename = None

# Train mode
encoder.train()
decoder.train()

# Start training
# In the training call, add hidden_size parameter
trainIters(model_config['model_name'], voc, pairs, encoder, decoder,
            encoder_optimizer, decoder_optimizer, embedding,
            model_config['encoder_n_layers'], model_config['decoder_n_layers'],
            'save', training_config['n_iteration'], model_config['batch_size'],
            training_config['print_every'], training_config['save_every'],
            training_config['clip'], 'corpus_name', loadFilename, device,
            hidden_size=model_config['hidden_size'])  # Added hidden_size

# Switch to evaluation mode
encoder.eval()
decoder.eval()

# Initialize searcher for evaluation
searcher = GreedySearchDecoder(encoder, decoder, device)

# Start chat
evaluateInput(encoder, decoder, searcher, voc)


Loading conversations...
Extracting pairs...
Read 221279 sentence pairs
Trimmed to 64239 sentence pairs
Counting words...
Counted words: 18066


/Users/koyiljonvaliev/Anaconda/anaconda3/envs/TorchEnv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Initializing...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 9.8033
Iteration: 2; Percent complete: 0.0%; Average loss: 9.6727
Iteration: 3; Percent complete: 0.1%; Average loss: 9.5123
Iteration: 4; Percent complete: 0.1%; Average loss: 9.1984
Iteration: 5; Percent complete: 0.1%; Average loss: 8.8623
Iteration: 6; Percent complete: 0.1%; Average loss: 8.0891
Iteration: 7; Percent complete: 0.2%; Average loss: 7.6779
Iteration: 8; Percent complete: 0.2%; Average loss: 7.3624
Iteration: 9; Percent complete: 0.2%; Average loss: 7.3177
Iteration: 10; Percent complete: 0.2%; Average loss: 7.0434
Iteration: 11; Percent complete: 0.2%; Average loss: 6.7065
Iteration: 12; Percent complete: 0.3%; Average loss: 6.4651
Iteration: 13; Percent complete: 0.3%; Average loss: 6.1287
Iteration: 14; Percent complete: 0.3%; Average loss: 6.0343
Iteration: 15; Percent complete: 0.3%; Average loss: 6.3460
Iteration: 16; Percent complete: 0.4%; Average loss: 5.7085
Iteration: 17; Percen